In [ ]:
# importing the necessary packages

from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [ ]:
DIRECTORY = r"dataset"
TYPES = ["with_mask", "without_mask"]

data = []                # initialize data list containing images
labels = []              # initialize labels list for labels of images

for i in TYPES:
    path = os.path.join(DIRECTORY, i)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(100, 100))
        image = img_to_array(image)
        data.append(image)
        labels.append(i)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
#print(labels.shape)
labels = to_categorical(labels)


In [ ]:
# Converting the data and labels in numpy array

data = np.array(data, dtype="float32")/255.0  # normalizing data
labels = np.array(labels)

In [ ]:
# Building the CNN Architecture of the model

model = Sequential()

model.add(Conv2D(200, (3, 3), input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# The first CNN layer followed by Relu activation and MaxPooling layer

model.add(Conv2D(100, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# The second CNN layer followed by Relu activation and MaxPooling layer

model.add(Flatten())    # Flatten layers to build fully connected layers from the outputs of second convolution layer
model.add(Dropout(0.5)) # Dropping Neurons with probability of 0.5

model.add(Dense(128, activation='relu'))    # Adding fully connected layer with 128 neurons

model.add(Dense(2, activation='softmax'))   # The Final output layer with two outputs for two categories

In [ ]:
# Summary of CNN Architecture of the model

model.summary()

In [ ]:
# Compilation of the Model

opt = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
# Splitting the data for Training and Testing 

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.1)

In [ ]:
# Creating Checkpoint for saving the Best Model

checkpoints = ModelCheckpoint('model-no-{epoch:02d}.model',monitor='val_loss',save_best_only=True,mode='auto')

In [ ]:
# Training the Model

FM = model.fit( trainX, trainY,
    callbacks=[checkpoints],validation_split=0.2,
    epochs=20)

In [ ]:
# Plotting Training and Validation Loss

plt.plot(FM.history['loss'],'g',label='training loss')
plt.plot(FM.history['val_loss'],label='validation loss')
plt.xlabel('No. of epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plotting Training and Validation Accuracy

plt.plot(FM.history['accuracy'],'g',label='training accuracy')
plt.plot(FM.history['val_accuracy'],label='validation accuracy')
plt.xlabel('No. of epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaulating the Trained Model

print(model.evaluate(testX,testY))